# 프로젝트: KoChatGPT 업그레이드 하기

### 정량적 분석을 위한 bleu 평가 함수 만들기

In [14]:
torch.cuda.empty_cache()

In [15]:
!pip install sacrebleu --q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
import pandas as pd
import numpy
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

In [17]:
from datasets import load_metric

# Load the BLEU metric
metric = load_metric("sacrebleu")

/tmp/ipykernel_162/3884723172.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [18]:
# Function to tokenize a sentence
def tokenize_sentence(sentence):
    # Implement tokenization using your tokenizer
    # Return a list of tokens
    tokens = tokenizer.tokenize(sentence)
    return tokens

In [19]:
# Function to calculate BLEU score
def calculate_bleu_score(model_outputs, target_sentences):
    # Tokenize model outputs and target sentences
    tokenized_outputs = [tokenize_sentence(output) for output in model_outputs]
    tokenized_targets = [tokenize_sentence(target) for target in target_sentences]
    
    # Calculate BLEU score
    bleu_score = metric.compute(predictions=tokenized_outputs, references=tokenized_targets)
    
    return bleu_score

### 평가를 위한 데이터셋 만들기 (효율을 위해 데이터셋 크기 줄이기 : 100개)

In [20]:
data_path_1_SFT = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file_SFT:
    list_data_dict_SFT = json.load(json_file_SFT)[11900:]

print(len(list_data_dict_SFT))
list_data_dict_SFT[:3]

100


[{'prompt': '플랫 화이트가 뭔가요?',
  'completion': '\'"플랫 화이트"는 일반적으로 흰색 배경에 만화나 일러스트 등을 담은 디자인을 의미합니다. 주로 웹툰, 게임, 애니메이션, 영화 등에서 사용되며, 평면적으로 표현된 그래픽 디자인의 스타일을 말합니다.',
  'tokens': 127},
 {'prompt': '어디에 김영삼의 서울대학교 입학 및 졸업 증명서가 전시되어 있는가?',
  'completion': "'김영삼의 서울대학교 입학 및 졸업 증명서는 서울대학교 미술관에 전시되어 있습니다.",
  'tokens': 105},
 {'prompt': '제가 지금 강남역인데, 여기서 오래 걸릴까요?',
  'completion': "'저는 인공지능 챗봇이므로 현재의 교통 상황과 여행 거리, 이동 방법 등과 같은 정보를 보유하고 있지 않습니다. 하지만, 일반적으로 강남역은 교통이 굉장히 복잡하고 혼잡하기 때문에 시간이 걸릴 수 있습니다. 만약 이동하실 위치가 어디인지 알려주신다면, 보다 더 정확한 답변을 드릴 수 있을 것입니다.",
  'tokens': 195}]

In [22]:
input_prompt = []
for i in list_data_dict_SFT:
    input_prompt.append(i['prompt'])
    
labeled_completion = []
for i in list_data_dict_SFT:
    labeled_completion.append(i['completion'])

# 2. SFT 모델과 RM 모델 결과 분석

## RM 적용 모델 결과 및 평가

In [23]:
### import modules
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [24]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

### 사용할 모델과 토크나이저를 불러오기

In [25]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')

tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=128,
)

with NaiveStrategy().model_init_context():
    model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()


Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### RM을 훈련시킬 때 사용할 ranking dataset을 만들기

In [26]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [27]:
total_data_ranking2chosen = []

for tmp in list_data_dict:
     prompt = tmp['prompt']
     ranking = tmp['ranking']

     for index in range(1, len(ranking)):
         n = ranking[0]
         m = ranking[index]


         data = {
             'prompt': prompt,
             'chosen': tmp['completion_{}'.format(n)],
             'rejected': tmp['completion_{}'.format(m)]
         }

         total_data_ranking2chosen.append(data)

In [28]:
import random
random.seed(12)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '콜라 마시고 리필 돼요?', 'chosen': '저는 AI 어시스턴트로 마시는 음료나 음식을 먹지 않기 때문에 정확한 답변을 드릴 수 없습니다. 하지만 대체로 맥도날드나 음식점 등에서는 콜라의 리필이 가능한 경우가 있습니다. 따라서 매장에 따라 다를 수 있으니, 참고하시기 바랍니다.', 'rejected': '다한국의 국제를 만드는 뜻이다. 한국에서 국제는 거의 국유 국換い中を 한국에서 국제를 이어나게 하는 개념이 국제들이 일어므는  믿을 개이 \n\n한국이 눇국폰 국제를 태는 아비 국동들의 국'}


In [29]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 128)
eval_dataset = RewardDataset(eval_data, tokenizer, 128)

1000
200




  0%|          | 0/1000 [00:00<?, ?it/s]

 14%|█▍        | 144/1000 [00:00<00:00, 1424.77it/s]

 31%|███       | 307/1000 [00:00<00:00, 1543.39it/s]

 46%|████▋     | 463/1000 [00:00<00:00, 1548.92it/s]

 63%|██████▎   | 629/1000 [00:00<00:00, 1591.05it/s]

 80%|████████  | 801/1000 [00:00<00:00, 1636.56it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1596.40it/s][A


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:00<00:00, 1659.83it/s]


In [30]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
내일 중요한 일이 있는데 옷 추천해주실 수 있나요?
######################################################################
## chosen ##
제가 AI 챗봇이기 때문에 옷 추천도 가능합니다! \n\n어느 정도의 포멀함인지, 혹은 어떤 색상을 선호하는지 등 세부적인 요구사항이 있으면 더 정확한 추천이 가능해집니다. \n\n그러나 일반적으로 중요한 일이라면 포멀한 범위에서 선택하기를 권해드리며, 어느 정도 안정감을 주는 색상 (ex. 검정, 회색, 네이비 등)의 옷을 추천해드립니다. \n\n무난하지만 세련된 느낌을 주는 옷을 선택하시는 것이 좋습니다. 행운을 빌어드립니다!
######################################################################
## rejected ##
죄송합니다. 옷 추천은 답변해 드릴 수 없습니다. 개인의 취향과 스타일에 따라 다르기 때문에 적합한 옷을 추천하기 어렵습니다. 하지만 다음과 같은 조언을 드릴 수 있습니다. 

1. 자신의 스타일과 취향을 고려하여 옷을 선택하세요. 
2. 자신의 색상과 가장 잘 어울리


### RM 학습

In [35]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=3,
                             max_epochs=1)

In [36]:
trainer.fit(use_lora=0)

model.save_pretrained('/output_2_RM')

Train epoch: 100%|██████████| 1/1 [01:11<00:00, 71.26s/it]


In [37]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -0.7


## PPO 적용

In [39]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [40]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=128
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [41]:
#optimizer
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [42]:
#tokenize
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [44]:
#train
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=5,  
                     train_batch_size=16, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [45]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

Train epoch [1/5]: 100%|██████████| 1/1 [00:00<00:00,  1.09it/s, actor_loss=0, critic_loss=0.00612]

Train epoch [2/5]: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s, actor_loss=0, critic_loss=0.378]

Train epoch [3/5]: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s, actor_loss=0, critic_loss=0.0616]

Train epoch [4/5]: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s, actor_loss=0, critic_loss=0.235]

Train epoch [1/5]: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s, actor_loss=0.306, critic_loss=0.115]

Train epoch [2/5]: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s, actor_loss=0.318, critic_loss=0.0159]

Train epoch [3/5]: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s, actor_loss=0.319, critic_loss=0.0466]

Train epoch [4/5]: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s, actor_loss=0.296, critic_loss=0.148]

Train epoch [1/5]: 100%|██████████| 1/1 [00:00<00:00,  1.06it/s, actor_loss=-.216, critic_loss=0.0717]

Train epoch [2/5]: 100%|██████████| 1/1 [00:00<00:00,  1.06it/s, actor_loss=-.22, c

In [46]:
model.save_pretrained('/output_3_PPO')

In [51]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=128,
                             do_sample=True,
                             top_k=50,
                             top_p=0.50,
                             no_repeat_ngram_size=4, 
                             early_stopping=True,
                             repetition_penalty=2.0,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]

    return output


In [53]:
RLHF_output = []
for input_text in input_prompt:
    new_output.append(generation(input_text))

In [54]:
# Calculate BLEU score
RLHF_bleu_score = []
for output, target in zip(RLHF_output, labeled_completion):
    RLHF_bleu_score.append(calculate_bleu_score([output], [target])['score'])

sum(RLHF_bleu_score)/len(RLHF_bleu_score)

0.2325450947266871

## RLHF 모델 bleu score : 0.232

### 답변 비교

In [64]:
input_prompt[1]

'어디에 김영삼의 서울대학교 입학 및 졸업 증명서가 전시되어 있는가?'

In [65]:
RLHF_output[1]

'어디에 김영삼의 서울대학교 입학 및 졸업 증명서가 전시되어 있는가? 뚱뚱 세인트 울프 [UNK]흩 진 訟 조교9 準 淮 기업 鳩 淮 기업 鳩 鳩쫌 人 [UNK] 진 訟 準 거래 鳩 淮 기업 鳩 효과 淮! 鳩쫌 人 [UNK] 쇠임 효과 淮!! 鳩쫌 鳩 효과 人 [UNK] 鳩 효과 淮!뙇임뙇르 鳩 효과 人 [UNK] 효과 仲 孫! 새로운 孫 鳩 효과 淮! 鳩 효과! 亦! 새로운 孫 鳩 효과 亨 [UNK] 鳩 효과 人 [UNK] 효과 仲 孫 孫! 새로운 孫 鳩 효과 淮! 鳩쫌 人 [UNK] 닝 숱 孫! 숱 孫 孫!'

## 2. SFT를 적용한 모델과 RLHF를 적용한 모델의 결과물을 정량/정성적으로 비교/분석
- 정량적 평가
    
    SFT 적용 모델의 bleu score : 0.137
    
    RLHF 모델의 bleu score : 0.232

    
    * 현재 두 모델 모두 output에 input의 prompt가 포함되었기 때문에 실제보다 더 높은 score가 나올 가능성이 있다. RLHF 모델의 bleu score가 더 낮게 나왔지만, 이는 정확하지 않은 지표라는 의문이 든다. 
        
        
- 정성적 평가
    
    SFT 모델의 평가 : 질문자의 질문에 관련성 있게 답변을 하였다. 질문자의 질문을 정확히 이해하고 맥락에 맞게 답변하였다. 다만, 대부분 질문에 대해서 정확하게 답변을 제안하지 못했다.
    
    
    RLHF 모델의 평가 : RLHF의 모델은 질문자의 질문과 답변 간의 관계가 거의 없다. 또한 반복된 단어를 사용하여 bleu score는 높은 값이 나왔지만, 질문과 관계없는 답변이 반복되어 챗봇의 기능으로서 도움이 되지 않는다. 단어를 나열하는 순서나 문장의 완성도 또한 올바르지 못했다.


    

# 3. Foundation Model 교체를 통한 정량적 성능향상

In [47]:
del model

In [48]:
import pandas as pd
import numpy
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = 'monologg/koelectra-base-v3-finetuned-korquad'
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                           bos_token='</s>', 
                                           eos_token='</s>', 
                                           unk_token='</s>', 
                                           pad_token='</s>',
                                           padding_side="right",
                                           model_max_length=128,
                                          )

If you want to use `ElectraForCausalLM` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at monologg/koelectra-base-v3-finetuned-korquad were not used when initializing ElectraForCausalLM: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing ElectraForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForCausalLM were not initialized from the model checkpoint at monologg/koelectra-base-v3-finetuned-korquad and are newly initialized: ['generator_predictions.dense.bias', 'generator_predictions.LayerNorm.bias', 'generator_lm_head.bias', 'generator_

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [49]:
new_output = []
max_length = 128

for input_txt in input_prompt:
    input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
    output= model.generate(input_ids, max_length=max_length, num_beams=4, no_repeat_ngram_size=4, early_stopping=True,
                                 eos_token_id=375, do_sample=True, top_k=50, repetition_penalty=2.0)
    new_output.append(tokenizer.decode(output[0]))

In [50]:
# Calculate BLEU score
new_bleu_score = []
for output, target in zip(new_output, labeled_completion):
    new_bleu_score.append(calculate_bleu_score([output], [target])['score'])
    
sum(new_bleu_score)/len(new_bleu_score)

Token indices sequence length is longer than the specified maximum sequence length for this model (130 > 128). Running this sequence through the model will result in indexing errors


0.2331180272431364

### monologg/koelectra-base-v3-finetuned-korquad을 적용한 모델의 bleu score : 0.233

## 분석 : 
 monologg/koelectra-base-v3-finetuned-korquad 모델을 사용하여 bleu score를 0.227에서 0.233까지 향상시켰다.

# 회고
잘한 점 : 코드에 대한 설명이 거의 없었음에도 다른 자료들을 활용하여 코드를 분석하고 주어진 미션에 맞게 고쳐 사용하였다.
    
못한 점 : 시간이 부족하여 다양한 전략을 사용해보지 못하고 간단한 것만 시도해본것이 아쉽다.
    
노력할 점 : bleu score 이외에도 정량적 평가를 할 수 있는 metrix를 사용해보면 더 좋을것 같다.